# Find the station you need and get the data formatted for analysis
* Find the dataset you need from the station list
* Download the dataset and unzip as a txt file
* Create csvs for each year in that file
* Clean those csvs up for use

In [1]:
# We need these libraries for this project
import pandas as pd
import numpy as np

## Step One
* Find the dataset you need from this [list](https://www1.ncdc.noaa.gov/pub/data/igra/igra2-station-list.txt)
* I have read that list in and turned it into a csv so we'll make that a searchable dataframe below
* Column [information](https://www1.ncdc.noaa.gov/pub/data/igra/igra2-list-format.txt) to help you find the station you need.

In [2]:
stations = pd.read_csv('igra2_station_list.csv', index_col=0)
stations.head()

,id,latitude,longitude,elevation,state,name,first_year,last_year,number_observations
0,ACM00078861,17.1170,-61.7830,10.0,,COOLIDGE FIELD (UA),1947,1993,13896
1,AEM00041217,24.4333,54.6500,16.0,,ABU DHABI INTERNATIONAL AIRPOR,1983,2020,37537
2,AEXUAE05467,25.2500,55.3700,4.0,,SHARJAH,1935,1942,2477
3,AFM00040911,36.7000,67.2000,378.0,,MAZAR-I-SHARIF,2010,2014,2179
4,AFM00040913,36.6667,68.9167,433.0,,KUNDUZ,2010,2013,4540


Put your own limitations on the dataframe to narrow it down to find the station ID that you need.  

Below I find the stations that offer current data for the state of California

In [3]:
# only want stations from the state of California
ca = stations[stations.state == 'CA']
# stations that have data available for 2020
ca = ca[ca.last_year == 2020]
ca

,id,latitude,longitude,elevation,state,name,first_year,last_year,number_observations
2121,USM00072293,32.8333,-117.1167,133.5,CA,SAN DIEGO/MIRAMAR,1946,2020,54295
2148,USM00072381,34.9167,-117.9000,705.0,CA,EDWARDS AFB,1942,2020,21432
2155,USM00072393,34.7500,-120.5667,121.0,CA,VANDENBERG AFB,1958,2020,43549
2196,USM00072493,37.7444,-122.2236,2.5,CA,OAKLAND/METROP. OAKLAND INT.,1928,2020,75950
2293,USM00074612,35.6833,-117.6833,696.0,CA,CHINA LAKE,1946,2020,12516


* Note the ID for the station that you want. I see that Oakland is USM00072493
* Find it in the [list](https://www1.ncdc.noaa.gov/pub/data/igra/data/) for the specific data set you need.
    * for data since January 1 go into 'data-y2d' and find your station id
    * 'data-por' contains data for the full period of record
* Download the file with your station ID as a zip
* Unzip the text file into the folder of your choice (the following notebook is set up assuming the file is located in the same folder as this notebook so change the file path appropriately if you store it elsewhere)
* You should now have a text file with your chosen station ID number in the name.

## Step 2
* Turn the text file you've downloaded into a csv for each year contained in that dataset
* They will be stored in the same folder as this notebook
* They will have the station ID and the year of the data as their name: 'USM00072493_2019.csv'
* Run the cell below to make the function available

In [4]:
def make_csv(filename): 
    '''(str)-> 'str'
    Input a file name for a NOAA IGRA data text file.  
    Returns 'Complete' when done.
    Creates a csv for each year of data that is in the input file.
    Does not clean or modify the data.
    '''
    # Open the file
    with open(filename) as f:
        # read in first line of data
        header = f.readline()   
        # if there's still data in the file keep reading it in
        while header:   
            # create a new file for each year of the data
            year = header[13:17]
            title = header[1:12] + '_' + year + '.csv'
            # create a list for each column 
            yr = []
            month = []
            day = []
            hour = []
            reltime = []
            p_src = []
            np_src = []
            lat = []
            long = []
            lvl_1 = []
            lvl_2 = []
            e_time = []
            pressure = []
            pflag = []
            gph = [] 
            zflag = []
            temp = [] 
            tflag = []
            rh = [] 
            dew_drop = [] 
            wdir = []
            wspd = []

            ''' 
            Each year gets it's own csv created, if it's the same year, we keep adding to the lists.
            If the year has changed it will kick you out to save the current lists as a csv
            then re enter the while loop above starting a new file for the new year.
            '''
            while year == header[13:17]:
                # each header row has data that's repeated for every sounding observation
                # save it once here and then add it for each row in the inner loop 
                n = int(header[32:36])   # number of observations under each header's information
                y = header[13:17]
                m = header[18:20]
                d = header[21:23]
                h = header[24:26]
                r = header[27:31]
                p = header[36:45]
                np = header[45:54]
                la = header[56:63]
                lo = header[63:71]

                # loop through each row of sounding data
                for i in range(n):
                    # read in a row of data
                    row = f.readline()
                    # add a value for each row from the header constants 
                    yr.append(y)
                    month.append(m)
                    day.append(d)
                    hour.append(h)
                    reltime.append(r)
                    p_src.append(p)
                    np_src.append(np)
                    lat.append(la)
                    long.append(lo)
                    # add this row's data to each column's list
                    lvl_1.append(row[0])
                    lvl_2.append(row[1])
                    e_time.append(row[3:8])
                    pressure.append(row[9:15])
                    pflag.append(row[15])
                    gph.append(row[16:21])
                    zflag.append(row[21])
                    temp.append(row[22:27])
                    tflag.append(row[27])
                    rh.append(row[28:33])
                    dew_drop.append(row[34:39])
                    wdir.append(row[40:45])
                    wspd.append(row[46:51])
                
                # read in the next header row to check if it's the same year at the top of the while loop
                header = f.readline()

            # make a dictionary when a year is complete 
            data = {'year': yr, 'month': month, 'day': day, 'hour':hour, 'rel_time':reltime, 'p_src': p_src,\
                    'np_src':np_src, 'lat': lat, 'long': long, 'lvl_1': lvl_1, 'lvl_2': lvl_2, \
                    'e_time':e_time, 'pressure': pressure, 'pflag': pflag,'gph':gph,\
                    'zflag':zflag, 'temp':temp, 'tflag':tflag, 'rh':rh, 'dew_drop':dew_drop, \
                    'wdir':wdir, 'wspd':wspd}
            # turn the dictionary into a dataframe
            df = pd.DataFrame(data)
            # save the dataframe as a csv and use the station ID and year for its name
            df.to_csv(title, index=False)
    
    # Reach the end of the text file      
    return "Complete"

Put the filename of the text file for the station's data that you want into the function call below then run that cell.    

I used Oakland in the example.

In [5]:
make_csv('USM00072493-data.txt')

'Complete'

### You now have a csv for each year contained in your dataset

## Step 3 
* Clean up the specific data you want to work with
* The files created above are all named with the station ID and the year of the observations 'ID_year.csv'
* We will get rid of -9999 and -8888 values and fill in some tags that are left blank
* We'll create some categorical columns and change speed and temp into more commonly used values
* You will need to do more specific cleaning for the type of analysis you want to do
* Column information [here](https://www1.ncdc.noaa.gov/pub/data/igra/data/igra2-data-format.txt)

Below is the function to create the cardinal wind direction column so run the cell if you want to use it.

In [6]:
def cardinal_wind(dir):
    if dir > 338 or dir <= 23:
        return 'N'
    elif dir > 23 and dir <= 68:
        return 'NE'
    elif dir > 68 and dir <= 113:
        return 'E'
    elif dir > 113 and dir <= 158:
        return 'SE'
    elif dir > 158 and dir <= 203:
        return 'S'
    elif dir > 203 and dir <= 248:
        return 'SW'
    elif dir > 248 and dir <= 293:
        return 'W'
    else:
        return 'NW'

In [7]:
def clean_igra(start, stop, station, name):
    '''(str) -> str
    Input the earliest year of the csvs you want to work with, the last year of data you want +1, 
    the station ID from the filenames, and the name you want to use for the clean files.
    
    Creates a csv with no blank values, and with standardised null values as np.NaN.
    Returns 'Complete' when done.
    
    Converts some units from tenths or hundreths so it's easier to understand the data.
    Adds a column for cardinal wind direction.
    Adds a column for pressure altitude in feet.
    Allows the option to convert speed to knots and height from meters to feet.
    
    clean_igra(1944, 2020, 'USM00072493', 'OAK')
        creates a cleaned csv labeled 'OAK_year.csv' for each year of data between 1944 and 2020.
    > 'Complete'
    '''
    # create a file for each year in the input range
    for year in range(start, stop):
        df = pd.read_csv(station + '_' + str(year) + '.csv')
        
        # get rid of 'blank' as a flag value for the three flag columns
        for col in ['pflag', 'zflag', 'tflag']:
            df[col] = df[col].apply(lambda x: 'C' if x == ' ' else x)
        
        # Turn -8888 and -9999 into NaN
        cols = ['pressure', 'gph', 'temp', 'rh', 'dew_drop', 'wdir', 'wspd', 'rel_time', 'e_time']
        for col in cols:
            df[col] = df[col].apply(lambda x: np.nan if (x == -9999 or x == -8888) else x)

        # Create a cardinal direction column
        df['cardinal'] = df.wdir.apply(cardinal_wind)
        
        # Pressure column in mb
        df.pressure = df.pressure.apply(lambda x: x/100)
        
        # Create a pressure altitude column
        df = df[~df['pressure'].isnull()]
        df['press_alt_ft'] = df.pressure.apply(lambda x: round(145366.45 * (1 - (x/1013.25)**.190284)))

        # Turn temp and percent humidity from tenths into full degrees C or %
        df.temp = df.temp.apply(lambda x: x/10)
        df.rh = df.rh.apply(lambda x: x/10)
        df.dew_drop = df.dew_drop.apply(lambda x: x/10)
        
        '''
        Uncomment lines of code provided below if you want your windspeed
        units to be knots, or your gph in feet.
        Beware you will introduce rounding error.
        '''
        # df.wspd = df.wspd.apply(lambda x: round((x/10) * 1.943844,2))
        # df.gph = df.gph.apply(lambda x: round(x/3.28084, 2)
        
        df.to_csv(name + '_' + str(year) + '.csv')
    
    # Have cleaned all CSVs requested
    return 'Complete'

Below I create one cleaned csv for the 2020 data.  We have three csvs, 2018, 2019, and 2020 created from the original text file above, but I only want to work with 2020 data in this example.  If I want all three files cleaned I simply change the first number to 2018 in the function call below.

In [8]:
clean_igra(2020, 2021, 'USM00072493', 'OAK')

'Complete'

## New Clean Columns Descriptions:
Original descriptions are [here](https://www1.ncdc.noaa.gov/pub/data/igra/data/igra2-data-format.txt)  
You can find what the abbreviations mean and the units used in the original observations.
* index:    The index 1 - number or rows
* month:    Still the month 1-12 of the sounding
* day:      Day of the month of the observation
* hour:     Hour of the observation UTC
* rel_time: release time of sounding UTC HHMM
* p_src:    Source of the pressure levels in the sounding.
* np_src    Source of the non-pressure levels in the sounding
* lat:      Latitude of sounding
* long:     Longitude of sounding
* lvl_1:    major level type indicator (1,2,3)
* lvl_2:    minor level type indicator (1,2,0)
* e_time:   elapsed time since launch MMMSS
* pressure: reported pressure in mb 
* pflag:    pressure flag (A,B,C)
* gph:      geopotential height in meters if you didn't uncomment the conversion code
    * geopotential height in ft above sea level (MSL pilot speak) if you used the conversion code
* zflag:    gph processing flag (A,B,C)
* temp:     temperature in degrees C
* tflag:    temp processing flag (A,B,C)
* rh:       relative humidity in percent
* dew_drop: dewpoint depression in degrees C
* wdir:     wind direction in degrees from north (0-359)
* wspd:     wind speed in tenths of meters per second if you didn't uncomment the conversion code
    * wind speed in knots if you did uncomment conversion code
* cardinal: wind direction in cardinal direction
* p_alt_ft: pressure altitude in feet


In [9]:
df = pd.read_csv('OAK_2020.csv', index_col=0)
df.head(10)

,year,month,day,hour,rel_time,p_src,np_src,lat,long,lvl_1,...,gph,zflag,temp,tflag,rh,dew_drop,wdir,wspd,cardinal,press_alt_ft
0,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,3.0,C,17.7,B,40.0,13.8,71.0,15.0,E,-178
1,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,162.0,B,15.1,B,39.6,13.6,205.0,6.0,SW,341
2,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,1,...,169.0,B,15.0,B,39.6,13.6,207.0,5.0,SW,364
3,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,526.0,B,12.7,B,41.2,12.8,304.0,24.0,NW,1530
4,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,1,...,824.0,B,13.1,B,30.5,16.9,348.0,87.0,N,2499
5,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,841.0,B,13.1,B,31.0,16.7,348.0,90.0,N,2553
6,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,1200.0,B,11.2,B,30.3,16.8,350.0,142.0,N,3715
7,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,1405.0,B,12.8,B,24.2,20.0,1.0,156.0,N,4374
8,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,1,...,1532.0,B,12.3,B,25.6,19.1,5.0,142.0,N,4779
9,2020,1,1,0,2300,ncdc-nws,ncdc-nws,377444,-1222236,2,...,1666.0,B,11.9,B,28.4,17.7,6.0,131.0,N,5209
